In [ ]:
import pandas as pd
import time
import datetime
pd.options.display.float_format = '{:,}'.format
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.float_format = '{:,}'.format

#https://docs.featuretools.com/en/stable/
import featuretools as ft
print('OK !')

In [ ]:
df = pd.read_csv('data/Data_Phoenix_Campaign_Enhancement.csv')

len(df)

In [ ]:
df['actual_offer_percent'] = df['campaign_premium']*100 / df['policy_annual_premium']
df['offer_score'] = df['campaign_offer_percent'] - df['actual_offer_percent']

In [ ]:
def change_dt(num):
    num = str(num)
    year = num[:4]
    mon = num[-2:]
    return datetime.datetime(int(year), int(mon), 1)
df['campaign_offer_date'] = df['campaign_offer_date'].astype(str)
df['campaign_offer_date'] = df['campaign_offer_date'].apply(change_dt)

In [ ]:
df['current_holding'] = df['campaign_offer_product'].map(str) + '_'+\
                    df['cus_ct_A1_product_holding'].map(str) + '_'+\
                    df['cus_ct_A2_product_holding'].map(str) + '_'+\
                    df['cus_ct_C1_product_holding'].map(str) + '_'+\
                    df['cus_ct_E1_product_holding'].map(str) + '_'+\
                    df['cus_ct_H1_product_holding'].map(str) + '_'+\
                    df['cus_ct_H2_product_holding'].map(str) 

prod_dict = {}
prod_dict['A1'] = 1
prod_dict['A2'] = 2
prod_dict['C1'] = 3
prod_dict['E1'] = 4
prod_dict['H1'] = 5
prod_dict['H2'] = 6

prod_dict

def is_existing(prod_text):
    prod_list = prod_text.split('_')
    prod_name = prod_list[0]
    
    cur_num = prod_list[prod_dict[prod_name]]
    
    return int(cur_num) 

df['current_holding'] = df['current_holding'].apply(is_existing)

In [ ]:
cols = ['policy_id','customer_age','customer_gender','cus_marital_status']
pol_df = df[cols].drop_duplicates()

cols = pol_df.columns
pol_df = pol_df.groupby('policy_id').first().reset_index()[cols]

pol_df.head()

In [ ]:
cols = ['policy_id','campaign_offer_date','policy_annual_premium','campaign_premium',
        'campaign_offer_percent','campaign_offer_product','actual_offer_percent','offer_score',
        'campaign_take_up',
        'current_holding'
        ]
offer_df = df[cols].drop_duplicates().reset_index()
offer_df=offer_df.rename(columns = {'index':'offer_id'})
offer_df.head()

In [ ]:
entities = {
       "policies" : (pol_df, "policy_id"),
       "offers" : (offer_df, "offer_id", "campaign_offer_date"),
    }

relationships = [("policies", "policy_id", "offers", "policy_id")]

ft.EntitySet("my-entity-set", entities, relationships).plot()

In [ ]:
es = ft.EntitySet("my-entity-set", entities, relationships)

In [ ]:
es["offers"]["campaign_take_up"].interesting_values = ["Y", "N"]
es["offers"]["campaign_offer_product"].interesting_values = ["E1", "A1", "A2", "C1","H1","H2"]

In [ ]:
start_time = time.time()

feature_matrix_policy, features_defs = ft.dfs(entities=es,  entityset=es,
                         agg_primitives=['max','median','trend','avg_time_between',\
                                         'min','std','mean','sum','skew',\
                                         'count','mode','last','time_since_first'
                                                                    ],
                        where_primitives=['max','median','trend','avg_time_between',\
                                         'min','std','mean','sum','skew',\
                                         'count','mode','last','time_since_first'
                                                                    ],
                                                 
                                         ignore_variables={'offers': ["offer_id"]},
                                                     target_entity="policies"
                                        )
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#feature_matrix_policy is compressed as feature_matrix_policy.csv.zip in data directory
feature_matrix_policy.to_csv('data/feature_matrix_policy.csv',index= False)

In [ ]:
feature_matrix_policy = pd.read_csv('data/feature_matrix_policy.csv')

In [ ]:
len(feature_matrix_policy.columns)
for each in feature_matrix_policy.columns:
    print(each)

In [ ]:
agg_ft = ft.primitives.list_primitives()
agg_ft[agg_ft['type'] == 'aggregation']